In [457]:
import pandas as pd
import re
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler
import nltk

nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [458]:
text = "Congressista dos EUA manda carta com cobrança a Moraes"

In [459]:
data = pd.DataFrame({
    'TEXT': [text],
    'RESULTADO': [None]
})

In [460]:
def remove_urls(text): #retira links que podem ter sido copiados, por meio da indicação do 'http'
    return re.sub(r'http\S+', '', text)

data['TEXT'] = data['TEXT'].apply(remove_urls)
data

,TEXT,RESULTADO
0,Congressista dos EUA manda carta com cobrança ...,None


In [461]:
def uppercase_percentage(text): #função que calcula a porcentagem de letras em maíscula no texto
    total_letters = sum(1 for char in text if char.isalpha())
    if total_letters == 0:
        return 0
    uppercase_count = sum(1 for char in text if char.isupper())
    percentage = (uppercase_count / total_letters)
    return percentage

In [462]:
data['uppercase_percentage'] = data['TEXT'].apply(uppercase_percentage) #segundo parâmetro que será utilizado para o treinamento da IA
data

,TEXT,RESULTADO,uppercase_percentage
0,Congressista dos EUA manda carta com cobrança ...,None,0.108696


In [463]:
def count_special_chars(text): #função que conta a quantidade de exclamações no texto
    special_chars = ['!']
    return sum(1 for char in text if char in special_chars)

In [464]:
data['special_char_count'] = data['TEXT'].apply(count_special_chars) #terceiro parâmetro utilizado no treinamento da IA
data

,TEXT,RESULTADO,uppercase_percentage,special_char_count
0,Congressista dos EUA manda carta com cobrança ...,None,0.108696,0


In [465]:
def remove_special_characters(text): #remove todos os caracteres especiais para ser possível fazer operações no texto corretamente
    pattern = r'[^\w\s]'
    clean_text = re.sub(pattern, ' ', text)
    return clean_text

In [466]:
data['TEXT'] = data['TEXT'].apply(remove_special_characters)
data

,TEXT,RESULTADO,uppercase_percentage,special_char_count
0,Congressista dos EUA manda carta com cobrança ...,None,0.108696,0


In [467]:
def remove_numbers(text): #remove os números da notícia para conseguir fazer operações no texto
    return re.sub(r'\b\w*\d\w*\b', '', text).strip()

In [468]:
data['TEXT'] = data['TEXT'].apply(remove_numbers)
data

,TEXT,RESULTADO,uppercase_percentage,special_char_count
0,Congressista dos EUA manda carta com cobrança ...,None,0.108696,0


In [469]:
fakeNewsList = ['lul', 'ment', 'faz', 'pod', 'tud', 'vai', 'terr', 'plan', 'brasil', 'bolsonar', 'tod', 'mai', 'mora']

In [470]:
def preprocessar_texto(texto): #stemiza, tokeniza e retira as stepwords.
    stemizador = RSLPStemmer() #stemização das palavras
    stop_words = set(stopwords.words('portuguese'))  # retirada das stepwords.

    # Tokenizar o texto
    palavras = word_tokenize(texto.lower())

    # Remover stopwords e aplicar stemização
    palavras_processadas = [
        stemizador.stem(palavra) for palavra in palavras
        if palavra.isalnum() and palavra not in stop_words
    ]

    return palavras_processadas


In [471]:
palavras_processadas = preprocessar_texto(text)

contagem = sum(palavra in fakeNewsList for palavra in palavras_processadas)
data['contagem_palavras_filtradas'] = contagem
data

,TEXT,RESULTADO,uppercase_percentage,special_char_count,contagem_palavras_filtradas
0,Congressista dos EUA manda carta com cobrança ...,None,0.108696,0,1


In [472]:
dataIA = data.drop(columns=['TEXT'])
dataIA

,RESULTADO,uppercase_percentage,special_char_count,contagem_palavras_filtradas
0,None,0.108696,0,1


In [473]:
from google.colab import drive
drive.mount('/content/drive') #conexão com o google drive (onde está o banco de dados)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [474]:
from sklearn.preprocessing import StandardScaler
from joblib import load

modelo_carregado = load('/content/drive/MyDrive/IA-projeto/aplicação/melhor_modelo.joblib')
scaler = load('/content/drive/MyDrive/IA-projeto/aplicação/scaler.pkl')
print("Modelo carregado com sucesso.")

# Preparar os dados
X = dataIA.drop(columns=['RESULTADO'])

XScaled = scaler.transform(X)

# Realizar as previsões
previsoes = modelo_carregado.predict(XScaled)

# Preencher a coluna 'RESULTADO' com as previsões
dataIA['RESULTADO'] = previsoes

XScaled

Modelo carregado com sucesso.


array([[-0.04286912, -0.30570892, -0.22190171]])

In [475]:
final = dataIA['RESULTADO'].loc[0]# 1 é news, 0 é fake

In [476]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# Carregar o modelo e tokenizer
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/IA-projeto/aplicação')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(torch.device('cpu'))

# Carregar os parâmetros salvos
model.load_state_dict(torch.load('/content/drive/MyDrive/IA-projeto/aplicação/melhor_modelo_bert.pkl', map_location=torch.device('cpu')))

# Configurar o modelo para avaliação
model.eval()

# Fazer previsões
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=256)
inputs = {key: val.to(torch.device('cpu')) for key, val in inputs.items()}

with torch.no_grad():
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    prediction = torch.argmax(outputs.logits, dim=1).item()

# Interpretação do resultado
if prediction == 1:
    print("Fake news")
else:
    print("News")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-476-ed9fea686663>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We re

News


In [477]:
final

1